In [2]:
# imports
from collections import defaultdict, Counter
import re

In [3]:
# utils
def get_input(day: int, test: bool = False):    
    with open(f"input/day{day}{'_test' if test else ''}.txt", "r") as f:
        return f.read()

def get_input_as_rows(day: int, test: bool = False):
    return get_input(day, test).split("\n")

def get_input_as_matrix(day: int, test: bool = False):
    return [list(row) for row in get_input_as_rows(day, test)]


In [4]:
# day 1
## part one
input_day1 = get_input_as_rows(1)
ll = [int(row.split(" ")[0].strip()) for row in input_day1]
rl = [int(row.split(" ")[-1].strip()) for row in input_day1]
ll.sort()
rl.sort()

total_distance = sum([abs(ll[i]- rl[i]) for i in range(len(ll))])
print(total_distance)

## part two
input_day1 = get_input_as_rows(1)
ll = [int(row.split(" ")[0].strip()) for row in input_day1]
rl = [int(row.split(" ")[-1].strip()) for row in input_day1]
c = Counter(rl)
similarity_score = sum([num * c[num] for num in ll])
print(similarity_score)


1830467
26674158


In [5]:
# day2
## part one
input_day2 = get_input_as_rows(2)
rows = [list(map(int, row.split(" "))) for row in input_day2]

safe_counter = 0
for row in rows:
    safe = True
    for i, num in enumerate(row):
        if i > 1 and ((num > row[i-1]) != (row[i-1] > row[i-2])):
            safe = False
            break
        if i != 0 and (abs(num - row[i-1]) > 3 or abs(num - row[i-1]) < 1):
            safe = False
            break
    if safe:
        safe_counter += 1
print(safe_counter)

## part two
input_day2 = get_input_as_rows(2)
rows = [list(map(int, row.split(" "))) for row in input_day2]

def is_safe(arr):
    safe = True
    for i, num in enumerate(arr):
        if i > 1 and ((num > arr[i-1]) != (arr[i-1] > arr[i-2])):
            safe = False
            break
        if i != 0 and (abs(num - arr[i-1]) > 3 or abs(num - arr[i-1]) < 1):
            safe = False
            break
    return safe

safe_counter = 0
for row in rows:
    if is_safe(row):
        safe_counter += 1
    else:
        for i in range(len(row)):
            if is_safe(row[:i] + row[i+1:]):
                safe_counter += 1
                break
    
print(safe_counter)



463
514


In [6]:
# day3
## part one
input_day3 = get_input(3)
pattern = r"mul\((\d{1,3}),(\d{1,3})\)"

matches = re.findall(pattern, input_day3)
result = sum([int(match[0]) * int(match[1]) for match in matches])
print(result)

## part two
mul_pattern = r"mul\((\d{1,3}),(\d{1,3})\)"
do_pattern = r"do\(\)"
dont_pattern = r"don't\(\)"

mul_matches = [(m.span(), int(m.group(1)), int(m.group(2))) for m in re.finditer(mul_pattern, input_day3)]
do_matches = [m.span()[0] for m in re.finditer(do_pattern, input_day3)]
dont_matches = [m.span()[0] for m in re.finditer(dont_pattern, input_day3)]

state_changes = [(pos, True) for pos in do_matches] + [(pos, False) for pos in dont_matches]
state_changes.sort()

enabled = True  # Start enabled
result = 0

for (start, end), num1, num2 in mul_matches:
    # Update enabled state based on any do/don't instructions before this mul
    for pos, new_state in state_changes:
        if pos < start:
            enabled = new_state
        else:
            break
            
    if enabled:
        result += num1 * num2

print(result)

185797128
89798695


In [7]:
# day4
## part one
input_day4 = get_input_as_matrix(4)

directions = [(0 ,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1), (-1,0), (-1,1)]
word = "XMAS"

def match(m, coord, direction, word, step):
    if step == len(word):
        return True
    if coord[0] < 0 or coord[0] >= len(m) or coord[1] < 0 or coord[1] >= len(m[0]):
        return False
    if word[step] == m[coord[0]][coord[1]]:
        return match(m, (coord[0] + direction[0], coord[1] + direction[1]), direction, word, step + 1)
    else:
        return False

match_counter = 0
for j, row in enumerate(input_day4):
    for i, char in enumerate(row):
        for direction in directions:
            if match(input_day4, (i, j), direction, word, 0):
                match_counter += 1
print(match_counter)
    
## part two
input_day4 = get_input_as_matrix(4)

match_counter = 0
for i, row in enumerate(input_day4):
    for j, char in enumerate(row):
        if char != "A":
            continue
        if any([
            match(input_day4, (i-1, j-1), (1,1), "MAS", 0),
            match(input_day4, (i-1, j-1), (1,1), "SAM", 0) 
        ]) and any([
            match(input_day4, (i+1, j-1), (-1,1), "MAS", 0),
            match(input_day4, (i+1, j-1), (-1,1), "SAM", 0) 
        ]):
            match_counter += 1
print(match_counter)



2462
1877


In [42]:
# day5
## part one
input_day5 = get_input_as_rows(5)
rules = defaultdict(list)
updates = []
for row in input_day5:
    if "|" in row:
        first, second = row.split("|")
        rules[int(first)].append(int(second))
    elif row != "":
        updates.append(list(map(int, row.split(","))))

def right_order(rules, update):
    seen = set()
    for number in update:
        for seen_number in seen:
            if seen_number in rules[number]:
                return False
        seen.add(number)
    return True

ret = 0
for update in updates:
    if right_order(rules, update):
        ret += update[len(update)//2]
print(ret)

## part two
rule_behind = defaultdict(list)
for row in input_day5:
    if "|" in row:
        first, second = row.split("|")
        rule_behind[int(second)].append(int(first))

def re_order(rule_behind, update):
    ptr = 0
    while ptr < len(update):
        for i in range(len(update)-1, ptr, -1):
            if update[i] in rule_behind[update[ptr]]:
                update = update[:ptr] + update[ptr+1:i] + [update[i]] + [update[ptr]] + update[i+1:]
                continue
        ptr += 1
    return update

ret = 0
for update in updates:
    if not right_order(rules, update):
        update = re_order(rule_behind, update)
        ret += update[len(update)//2]
print(ret)






6267
5184
